<a href="https://colab.research.google.com/github/AkashCreator/Pyspark/blob/main/Pyspark_Football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
# install pyspark
!pip install pyspark

In [106]:
# read dataset using Sparksession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("football").enableHiveSupport().getOrCreate()

In [107]:
#display spark session
spark

In [108]:
#read football main dataset and show stats
full_data=spark.read.csv('./football_analysis/FullData.csv',header=True,inferSchema=True)
full_data.summary().show()

+-------+----------------+-----------+-----------------+-----------------+----------------+-------------+------------------+------------+------------------+-----------------+------+------+--------------+----------+------------------+------------------+---------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+
|summary|            Name|Nationality|National_Positio

# Below are some of the observations made after running summary command

  
1.   There is one entry where we dont have data for Club_Position and some other parameters
2.   Minimum age of the player is 17(kinda expected) and max age is 47 which is bit odd
3.   Minimum weight is coming as 100 kg and Maximum weight is coming as 99 kg which is absurd(so more analysis is required here to identify rootcause)
4.   Contract expiry year is coming as Float so needs some formatting
5.   Birthdate is integer not a date format

In [109]:
#read sample data
full_data.sample(withReplacement=False,fraction=0.1).show()

+------------------+-----------+-----------------+------------+---------------+-------------+--------+------------+---------------+------+------+------+--------------+----------+---+------------------+---------------+---------+-----------+------------+---------+-------+--------------+---------------+----------+---------+------------------+-------------+------+---------+--------+----------+---------+------------+-----+-------+--------+-------+-------+-------+-------+----------+---------+----------+-----+-----------------+---------+-------+--------------+---------+----------+-----------+-----------+
|              Name|Nationality|National_Position|National_Kit|           Club|Club_Position|Club_Kit|Club_Joining|Contract_Expiry|Rating|Height|Weight|Preffered_Foot|Birth_Date|Age|Preffered_Position|      Work_Rate|Weak_foot|Skill_Moves|Ball_Control|Dribbling|Marking|Sliding_Tackle|Standing_Tackle|Aggression|Reactions|Attacking_Position|Interceptions|Vision|Composure|Crossing|Short_Pass|Lon

In [110]:
#display schema
full_data.dtypes

[('Name', 'string'),
 ('Nationality', 'string'),
 ('National_Position', 'string'),
 ('National_Kit', 'double'),
 ('Club', 'string'),
 ('Club_Position', 'string'),
 ('Club_Kit', 'double'),
 ('Club_Joining', 'string'),
 ('Contract_Expiry', 'double'),
 ('Rating', 'int'),
 ('Height', 'string'),
 ('Weight', 'string'),
 ('Preffered_Foot', 'string'),
 ('Birth_Date', 'string'),
 ('Age', 'int'),
 ('Preffered_Position', 'string'),
 ('Work_Rate', 'string'),
 ('Weak_foot', 'int'),
 ('Skill_Moves', 'int'),
 ('Ball_Control', 'int'),
 ('Dribbling', 'int'),
 ('Marking', 'int'),
 ('Sliding_Tackle', 'int'),
 ('Standing_Tackle', 'int'),
 ('Aggression', 'int'),
 ('Reactions', 'int'),
 ('Attacking_Position', 'int'),
 ('Interceptions', 'int'),
 ('Vision', 'int'),
 ('Composure', 'int'),
 ('Crossing', 'int'),
 ('Short_Pass', 'int'),
 ('Long_Pass', 'int'),
 ('Acceleration', 'int'),
 ('Speed', 'int'),
 ('Stamina', 'int'),
 ('Strength', 'int'),
 ('Balance', 'int'),
 ('Agility', 'int'),
 ('Jumping', 'int'),
 ('He

First we will analyse the weight data

In [111]:
#As expected we are getting min as 100 because of kg in the column data so we will remove it to allow better analysis
full_data.select('Weight').distinct().orderBy('Weight',ascending=False).show()

+------+
|Weight|
+------+
| 99 kg|
| 98 kg|
| 97 kg|
| 96 kg|
| 95 kg|
| 94 kg|
| 93 kg|
| 92 kg|
| 91 kg|
| 90 kg|
| 89 kg|
| 88 kg|
| 87 kg|
| 86 kg|
| 85 kg|
| 84 kg|
| 83 kg|
| 82 kg|
| 81 kg|
| 80 kg|
+------+
only showing top 20 rows



In [112]:
#import sql functions from pyspark library
#replace ' kg' string to empty
#type string data to Int and then display the sorted data to ensure its formatted correctly
from pyspark.sql.functions import *
full_data=full_data.withColumn('Weight',regexp_replace('Weight',' kg',''))
full_data=full_data.withColumn('Weight',full_data.Weight.cast('Integer'))\
.orderBy('Weight',ascending=False)

# "Above weight data confirms that formatting has been done correctly and now can be used for any analysis/aggregates"

In [113]:
#print schema to check nullable columns.One observation is birthdate is string rather than date format
#and to ensure weight is now Int
full_data.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- National_Position: string (nullable = true)
 |-- National_Kit: double (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club_Position: string (nullable = true)
 |-- Club_Kit: double (nullable = true)
 |-- Club_Joining: string (nullable = true)
 |-- Contract_Expiry: double (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Preffered_Foot: string (nullable = true)
 |-- Birth_Date: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Preffered_Position: string (nullable = true)
 |-- Work_Rate: string (nullable = true)
 |-- Weak_foot: integer (nullable = true)
 |-- Skill_Moves: integer (nullable = true)
 |-- Ball_Control: integer (nullable = true)
 |-- Dribbling: integer (nullable = true)
 |-- Marking: integer (nullable = true)
 |-- Sliding_Tackle: integer (nullable = true)
 |-- Standing_Tackle

In [114]:
#count the no of clubs just to ensure data is expected
full_data.select('Club').distinct().count()

634

In [115]:
#cast the Date column
full_data=full_data.withColumn('Birth_Date',full_data.Birth_Date.cast('Date'))
full_data.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- National_Position: string (nullable = true)
 |-- National_Kit: double (nullable = true)
 |-- Club: string (nullable = true)
 |-- Club_Position: string (nullable = true)
 |-- Club_Kit: double (nullable = true)
 |-- Club_Joining: string (nullable = true)
 |-- Contract_Expiry: double (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Height: string (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Preffered_Foot: string (nullable = true)
 |-- Birth_Date: date (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Preffered_Position: string (nullable = true)
 |-- Work_Rate: string (nullable = true)
 |-- Weak_foot: integer (nullable = true)
 |-- Skill_Moves: integer (nullable = true)
 |-- Ball_Control: integer (nullable = true)
 |-- Dribbling: integer (nullable = true)
 |-- Marking: integer (nullable = true)
 |-- Sliding_Tackle: integer (nullable = true)
 |-- Standing_Tackle: 

In [116]:
!pip install plotly

In [122]:
# save data as hive table
data1=full_data.write.saveAsTable('footballdata')

In [125]:
# show databases
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [126]:
# show tables
spark.sql("show tables").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|  default|footballdata|      false|
+---------+------------+-----------+



In [146]:
# Find the top 20 countries which produce the highest no of players
data_sql=spark.sql("""SELECT nationality,count(*)
                               AS Total_Players
                                  FROM footballdata
                                    GROUP BY nationality
                                       ORDER BY count(*) DESC,nationality LIMIT 20 """)
data_sql.show()

+-------------------+-------------+
|        nationality|Total_Players|
+-------------------+-------------+
|            England|         1618|
|          Argentina|         1097|
|              Spain|         1008|
|             France|          974|
|             Brazil|          921|
|              Italy|          751|
|            Germany|          689|
|           Colombia|          592|
|              Japan|          471|
|Republic of Ireland|          442|
|        Netherlands|          426|
|              Chile|          398|
|             Sweden|          378|
|           Portugal|          360|
|       Saudi Arabia|          354|
|            Denmark|          342|
|             Norway|          342|
|             Mexico|          341|
|      United States|          332|
|             Poland|          328|
+-------------------+-------------+



In [150]:
# Find the top 20 countries which produce the highest no of players
data_sql_penalties=spark.sql("""SELECT nationality,sum(penalties)
                               AS Total_Penalties
                                  FROM footballdata
                                    GROUP BY nationality
                                       ORDER BY sum(penalties) DESC,nationality LIMIT 20 """)
data_sql_penalties.show()

+-------------------+---------------+
|        nationality|Total_Penalties|
+-------------------+---------------+
|            England|          77059|
|          Argentina|          56369|
|              Spain|          52113|
|             Brazil|          50937|
|             France|          48405|
|              Italy|          36183|
|            Germany|          33095|
|           Colombia|          28605|
|        Netherlands|          21163|
|              Japan|          20549|
|Republic of Ireland|          20500|
|              Chile|          18886|
|           Portugal|          18776|
|             Mexico|          17648|
|             Sweden|          16936|
|             Norway|          15728|
|       Saudi Arabia|          15424|
|      United States|          15273|
|             Russia|          15259|
|             Poland|          15141|
+-------------------+---------------+



In [153]:
# join the two datasets using inner join
data_sql_merge=data_sql.join(data_sql_penalties,how="inner",on="nationality")

In [155]:
# sort the data on counts
data_sql_merge.orderBy(['Total_Players','Total_Penalties'],ascending=False).show()

+-------------------+-------------+---------------+
|        nationality|Total_Players|Total_Penalties|
+-------------------+-------------+---------------+
|            England|         1618|          77059|
|          Argentina|         1097|          56369|
|              Spain|         1008|          52113|
|             France|          974|          48405|
|             Brazil|          921|          50937|
|              Italy|          751|          36183|
|            Germany|          689|          33095|
|           Colombia|          592|          28605|
|              Japan|          471|          20549|
|Republic of Ireland|          442|          20500|
|        Netherlands|          426|          21163|
|              Chile|          398|          18886|
|             Sweden|          378|          16936|
|           Portugal|          360|          18776|
|       Saudi Arabia|          354|          15424|
|             Norway|          342|          15728|
|           